In [14]:
// Parse Input
using Newtonsoft.Json;
using System.Numerics;

/*
var input = System.IO.File.ReadAllLines("sample-input.txt");
/*/ 
var input = System.IO.File.ReadAllLines("input.txt");
//*/


In [2]:
IEnumerable<List<string>> Monkeys(string[] v){
    var section = new List<string>();
    foreach(var line in v){
        section.Add(line);
        if(string.IsNullOrEmpty(line)){
            yield return section;
            section = new List<string>();
        }
    }

    if(section.Any()) yield return section;
}

## Part 1

In [ ]:

public class Monkey {
    public Monkey(List<string> initialState){
        items = initialState[1].Split(':')[1].Split(',').Select(i => int.Parse(i)).ToList();
        operation = initialState[2].Split('=')[1].Trim();
        divisibleNumber = int.Parse(initialState[3].Split(' ').Last());
        throwToMonkeyTrue = int.Parse(initialState[4].Split(' ').Last());
        throwToMonkeyFalse = int.Parse(initialState[5].Split(' ').Last());
        itemsInspected = 0;
    }

    public List<int> items {get;set;}
    public string operation {get;set;}
    public int divisibleNumber {get;set;}
    public int throwToMonkeyTrue {get;set;}
    public int throwToMonkeyFalse {get;set;}
    public int itemsInspected {get;set;}

    public void CatchItem(int item){
        items.Add(item);
    }
}

public static void InspectsItems(this Monkey monkey, List<Monkey> gang){
    foreach(var item in monkey.items){
        var @new = Convert.ToInt32(new System.Data.DataTable().Compute(monkey.operation.Replace("old", $"{item}"),null));
        @new = @new/3;
        if(@new % monkey.divisibleNumber == 0){
            gang[monkey.throwToMonkeyTrue].CatchItem(@new);
        }else{
            gang[monkey.throwToMonkeyFalse].CatchItem(@new);
        }
        monkey.itemsInspected++;
    }
    monkey.items = new List<int>();
}


In [ ]:
var monkeys = Monkeys(input).Select(m => new Monkey(m)).ToList();

for(var i = 0; i < 20; i++){
    foreach(var monkey in monkeys){
        monkey.InspectsItems(monkeys);
    }
    //Console.WriteLine(JsonConvert.SerializeObject(monkeys));
}

var top2Monkeys = monkeys.OrderByDescending(x => x.itemsInspected).Take(2);

top2Monkeys.Select(x => x.itemsInspected).Aggregate((a, x) => a * x)


## Part 2

In [18]:
public class Item {
    public Item(int m, string i){
        monkey = m;
        worryLevel = new BigInteger(int.Parse(i));
    }

    public BigInteger worryLevel {get;set;}
    public int monkey {get;set;}
}

static var items = new List<Item>();

static long sudoModulo = 0;

public class MonkeyV2 {
    public MonkeyV2(List<string> initialState){
        monkeyNumber = int.Parse(initialState[0].Split(' ').Last().Replace(":",""));
        items.AddRange(initialState[1].Split(':')[1].Split(',').Select(i => new Item(monkeyNumber,i)));
        operation = initialState[2].Split('=')[1].Trim();
        divisibleNumber = int.Parse(initialState[3].Split(' ').Last());
        throwToMonkeyTrue = int.Parse(initialState[4].Split(' ').Last());
        throwToMonkeyFalse = int.Parse(initialState[5].Split(' ').Last());
        itemsInspected = 0;
    }

    public int monkeyNumber {get;set;}
    public string operation {get;set;}
    public int divisibleNumber {get;set;}
    public int throwToMonkeyTrue {get;set;}
    public int throwToMonkeyFalse {get;set;}
    public long itemsInspected;

    public async Task InspectItems(){
        var monkeyItems = items.Where(x => x.monkey == monkeyNumber).Select(x => InspectItem(x));
        await Task.WhenAll(monkeyItems);       
    }

    Task InspectItem(Item item){
        var op = operation.Split(' ');
        var a =  op[0] == "old" ? item.worryLevel : new BigInteger(long.Parse(op[0]));
        var b =  op[2] == "old" ? item.worryLevel : new BigInteger(long.Parse(op[2]));

                 
        var value = op[1] switch {
            "+" => a + b,
            "*" => a * b
        };

        item.worryLevel = value % sudoModulo;

        if(item.worryLevel % divisibleNumber == 0){
            item.monkey = throwToMonkeyTrue;
        }else{
            item.monkey = throwToMonkeyFalse;
        }

        System.Threading.Interlocked.Increment(ref itemsInspected);            

        return Task.CompletedTask;
    }
}

In [19]:
items = new List<Item>();
var monkeys = Monkeys(input).Select(m => new MonkeyV2(m)).ToList();

sudoModulo = monkeys.Select(x => x.divisibleNumber).Aggregate((a, x) => a * x);

for(var i = 0; i < 10000; i++){
   // foreach(var group in items.GroupBy(x => x.monkey).OrderBy(x => x.Key)){
   //     Console.WriteLine($"Monkey {group.Key}: {string.Join(',', group.Select(x => x.worryLevel))}");
   // }
    foreach(var monkey in monkeys){
        await monkey.InspectItems();
    }
}

monkeys.Select(x => x.itemsInspected)


index,value
0,115025
1,93029
2,114545
3,114164
4,8590
5,30465
6,122693
7,122667


In [20]:



var top2Monkeys = monkeys.OrderByDescending(x => x.itemsInspected).Take(2);

top2Monkeys.Select(x => x.itemsInspected).Aggregate((a, x) => a * x)

15050382231